## Create FHIR R4 CapStatement Resource

### Outline:

- Source excel with requirements
- pandas to convert in python Ordered Dict
- build json
- generate narrative using Jinja2 templates

### Prerequisites:

- Python 3.6 or greater

### Import FHIRClient and other libraries

In [140]:
%config IPCompleter.greedy=True

In [141]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
from fhirclient.r4models import narrative as N
import fhirclient.models.identifier as I
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime, date
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from itertools import zip_longest
from openpyxl import load_workbook
from commonmark import commonmark
from lxml import etree

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the canonical base and project information

In [142]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
pre = "US-Core"
canon = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        )
ig_package_path =  "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/output"
#ig_package_path =  "/Users/ehaas/Documents/FHIR/US-Core-R4/output"
#ig_package_path = "C:/Users/Administrator/Downloads/"
#ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = "/Users/ehaas/Documents/FHIR/US-Core-R4/source/"
ig_source_path = ''

# spreadsheet source
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path ='../SP/'
#in_path =''
in_file ="uscore-server"
in_file ="uscore-client"
#******************** Need to update *************************************************

f_jurisdiction =  CC.CodeableConcept({
      "coding" : [
        {
          "system" : "urn:iso:std:iso:3166",
          "code" : "US"
        }
      ]
    })

conf_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation'
combo_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-search-parameter-combination'

none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False']

sep_list = (',', ';', ' ', ', ', '; ')

f_now = D.FHIRDate(str(date.today()))
f_now.as_json()

'2019-05-14'

#### Conformance Extension

In [143]:
def get_conf(conf='MAY'):
    return [X.Extension(dict(
        url = conf_url,
        valueCode = conf
        ))]

### validate

In [144]:
# *********************** validate Resource ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/Questionnaire/$validate', params = params, headers = headers, data = dumps(r.as_json()))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Cap Statement input data

#### first the meta sheet

In [145]:
xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'meta',na_filter = False)

df

,Element,Value
0,id,us-core-client
1,description,​The Section describes the expected capabiliti...
2,ig,http://hl7.org/fhir/us/core/ImplementationGuid...
3,mode,client
4,documentation,The US Core Client **SHALL**:\n\n1. Support fe...
5,security,1. See the [General Security Considerations] s...


#### Create NamedTuple from df to use dot notation

In [146]:
d = dict(zip(df.Element, df.Value))
meta = namedtuple("Meta", d.keys())(*d.values())      
         
meta.id

'us-core-client'


### Create CS instance

In [147]:
def get_op():
    op_list = []
    df_op = read_excel(xls,'ops',na_filter = False)
    for i in df_op.itertuples(index=True):
        op = CS.CapabilityStatementRestResourceOperation()
        op.name = i.name 
        op.definition = i.definition
        op.extension = get_conf(i.conf) 
        op_list.append(op)
    return op_list

def get_igs():
    ig_list = []
    df_igs = read_excel(xls,'igs',na_filter = False)
    for ig in df_igs.itertuples(index=True):
        ig_list.append(ig.uri)
    return ig_list

def kebab_to_pascal(word):
    return ''.join(x.capitalize() for x in word.split('-'))

cs = CS.CapabilityStatement()
cs.id = meta.id
cs.url = f'{canon}CapabilityStatement/{meta.id}'
cs.version = '0.0.0'  # placeholder changed by build
cs.name = f'{kebab_to_pascal(meta.id)}{cs.resource_type}'
cs.title = f'{titlecase(meta.id)} {cs.resource_type}'
cs.status = 'active'
cs.experimental = False
cs.date = f_now  # as FHIRDate
cs.publisher = publisher
cs.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
cs.description = meta.description
cs.jurisdiction = [f_jurisdiction]
cs.kind = 'requirements'
cs.fhirVersion = '4.0.0'
cs.acceptUnknown = 'both'
cs.format = [
    "xml",
    "json"
  ]
cs.patchFormat = [
    "application/json-patch+json",
  ]
cs.implementationGuide = meta.ig.split(",") + get_igs()
rest = CS.CapabilityStatementRest(dict(
    mode = meta.mode,
    documentation = meta.documentation,
    security = dict(
        description = meta.security
        )
    ))
cs.rest = [rest]


cs.as_json()

{'id': 'us-core-client',
 'contact': [{'telecom': [{'system': 'url',
     'value': 'http://www.hl7.org/Special/committees/structure/index.cfm'}]}],
 'date': '2019-05-14',
 'description': '\u200bThe Section describes the expected capabilities of the US Core Client which is responsible for creating and initiating the queries for information about an individual patient. The complete list of FHIR profiles, RESTful operations, and search parameters supported by US Core Servers are defined in the [Conformance Requirements for Server](CapabilityStatement-us-core-server.html). US Core Clients have the option of choosing from this list to access necessary data based on their local use cases and other contextual requirements.\n\n*NOTE* : The [capabilitystatement-expectation](http://hl7.org/fhir/R4/extension-capabilitystatement-expectation.html) extension is used throughout this resource to formally express implementer expectations for the various elements.   This guide extends the usage context 

#### Then the list of IG profiles

In [148]:
xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'profiles',na_filter = False)

df

,Profile,Name,Conformance,Type
0,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core CareTeam Profile,SHOULD,CareTeam
1,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core MedicationStatement Profile,SHOULD,MedicationStatement
2,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core Device Profile,SHOULD,Device
3,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core Practitioner Profile,SHOULD,Practitioner
4,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core Patient Profile,SHOULD,Patient
5,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core Immunization Profile,SHOULD,Immunization
6,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core Smoking Status Observation Profile,SHOULD,Observation
7,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core DocumentReference Profile,SHOULD,DocumentReference
8,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core MedicationRequest Profile,SHOULD,MedicationRequest
9,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core Condition Profile,SHOULD,Condition


#### add Resources

- read sheets for reasource attributes, interaction attributes,  search attributes, profiles, and combo search parameters

In [149]:
df_resources = read_excel(xls,'resources',na_filter = False)
df_profiles = read_excel(xls,'profiles',na_filter = False)
df_i = read_excel(xls,'interactions',na_filter = False)
df_sp = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)
df_op = read_excel(xls,'ops',na_filter = False)


def get_i(type):
    int_list = []
    for i in df_i.itertuples(index=True):
        #print(i.code, getattr(i,f'conf_{type}'))
        if getattr(i,f'conf_{type}') not in none_list:
            int  = CS.CapabilityStatementRestResourceInteraction()
            int.code = i.code
            try:
                int.documentation = getattr(i,f'doc_{type}')
            except:
                pass
            int.extension = get_conf(getattr(i,f'conf_{type}'))    
            int_list.append(int.as_json())
        
    return int_list


def get_sp(r_type):
    sp_list = []
    for i in df_sp.itertuples(index=True):
        if i.base == r_type:
            sp  = CS.CapabilityStatementRestResourceSearchParam()
            sp.name = i.code
            
            # TODO need to fix this to reference a mapping see SP directory
       
            sp.definition = (f'{canon}SearchParameter/{pre.lower()}-{i.base.lower()}-{i.code.split("_")[-1]}' if i.update == 'Y' or i.exists =='N'
                             else f'{fhir_base_url}SearchParameter/{i.base}-{i.code.split("_")[-1]}')  # removes the '_' for things like _id
            # print(sp.definition)
            sp.type = i.type
            sp.extension = get_conf(i.base_conf)
            #print(sp.as_json())                
            sp_list.append(sp.as_json())
                             
    return sp_list


def get_combo_ext(r_type,combos):
    x_list = []
    for combo in combos:
        # convert to extension
        combo_ext = X.Extension()
        combo_ext.url = combo_url
        combo_conf_ext = get_conf(combo[1])
        combo_ext.extension=combo_conf_ext
        for param in combo[0].split(','):
            req_combo = X.Extension(
                dict (
                    url = 'required',
                    valueString = param   #http://hl7.org/fhir/us/core/SearchParameter/us-core-patient-family
                    )
                )
            combo_ext.extension.append(req_combo)
        x_list.append(combo_ext)
        # print(x_list)
    return x_list
                             
def get_op(r_type):
    op_list = []
    for i in df_op.itertuples(index=True):
         if i.type == r_type:
            op = CS.CapabilityStatementRestResourceOperation()
            op.name = i.name 
            op.definition = i.definition
            op.documentation = i.documentation
            op.extension = get_conf(i.conf) 
            op_list.append(op.as_json())
    return op_list 

rest.resource =  []
for r in df_resources.itertuples(index=True):
    # print(r.type, r.conformance, r.readHistory)
    supported_profile = [p.Profile for p in df_profiles.itertuples(index=True) if p.Type == r.type]
    #pprint(supported_profile)                         
    res = CS.CapabilityStatementRestResource(
    dict(
        type = r.type,
        documentation = r.documentation if r.documentation not in none_list else None,
        versioning = r.versioning if r.versioning not in none_list else None,
        readHistory = r.readHistory if r.readHistory not in none_list else None,
        updateCreate = r.updateCreate if r.updateCreate not in none_list else None,
        conditionalCreate = r.conditionalCreate if r.conditionalCreate not in none_list else None,
        conditionalRead = r.conditionalRead if r.conditionalRead not in none_list else None,
        conditionalUpdate = r.conditionalUpdate if r.conditionalUpdate not in none_list else None,
        conditionalDelete = r.conditionalDelete if r.conditionalDelete not in none_list else None,
        referencePolicy = [x for x in r.referencePolicy.split(",") if x],
        searchInclude =  [x for x in r.shall_include.split(",") + r.should_include.split(",") if x],
        searchRevInclude =  [x for x in r.shall_revinclude.split(",") + r.should_revinclude.split(",") if x],
        interaction = get_i(r.type),
        searchParam = get_sp(r.type),
        operation = get_op(r.type),
        profile = f'{fhir_base_url}StructureDefinition/{r.type}',
        supportedProfile = supported_profile,
        )
    )
    res.extension = get_conf(r.conformance)
    combos = {(i.combo,i.combo_conf) for i in df_combos.itertuples(index=True) if i.base == r.type}
    res.extension = res.extension + get_combo_ext(r.type,combos) # convert list to  lst of combo extensions
                             
    rest.resource.append(res)
cs.rest = [rest]
    
print(dumps(cs.as_json(),indent=3))    
        
        

{
   "id": "us-core-client",
   "contact": [
      {
         "telecom": [
            {
               "system": "url",
               "value": "http://www.hl7.org/Special/committees/structure/index.cfm"
            }
         ]
      }
   ],
   "date": "2019-05-14",
   "description": "\u200bThe Section describes the expected capabilities of the US Core Client which is responsible for creating and initiating the queries for information about an individual patient. The complete list of FHIR profiles, RESTful operations, and search parameters supported by US Core Servers are defined in the [Conformance Requirements for Server](CapabilityStatement-us-core-server.html). US Core Clients have the option of choosing from this list to access necessary data based on their local use cases and other contextual requirements.\n\n*NOTE* : The [capabilitystatement-expectation](http://hl7.org/fhir/R4/extension-capabilitystatement-expectation.html) extension is used throughout this resource to formall

### Convert model to dict and add extensions to primitives

In [150]:
#cs = cs.as_json()

### Validate

In [151]:
 #validate and write to file

print('...validating')
r = validate(cs)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))



...validating


### Create Narrative

- Using Jinja2 Template create xhtml for narrative

#### First: Get spec_internal from package.tgz a json file which includes canonical to local relative page links

Note for this to work you have to have a working build that already contains all the needed artifacts.

In [152]:
import tarfile
def get_si(path):
    with tarfile.open(name=os.path.join(path,'package.tgz'), mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('other/spec.internals')
        r = f.read()
        return(loads(r))
        
si = get_si(ig_package_path)
path_map = si['paths']
path_map

{'http://hl7.org/fhir/us/core/ImplementationGuide/hl7.fhir.us.core.r4-4.0.0|4.0.0': '4.0.0/ImplementationGuide-hl7.fhir.us.core.r4-4.0.0.html',
 'http://hl7.org/fhir/us/core/ImplementationGuide/hl7.fhir.us.core.r4-4.0.0': 'ImplementationGuide-hl7.fhir.us.core.r4-4.0.0.html',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest|4.0.0': '4.0.0/StructureDefinition-us-core-medicationrequest.html',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest': 'StructureDefinition-us-core-medicationrequest.html',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal|4.0.0': '4.0.0/StructureDefinition-us-core-goal.html',
 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal': 'StructureDefinition-us-core-goal.html',
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height|4.0.0': '4.0.0/StructureDefinition-pediatric-weight-for-height.html',
 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-heigh

#### Then Use Jinja2 template to create narrative

In [153]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)



env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown


template = env.get_template(in_file)

sp_map = {sp.code:sp.type for sp in df_sp.itertuples(index=True)}
pname_map = {p.Profile:p.Name for p in df_profiles.itertuples(index=True)}

rendered = template.render(cs=cs, path_map=path_map, pname_map = pname_map, sp_map =sp_map )

display(HTML(rendered))


parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))
narr = N.Narrative()
narr.status = 'generated'
narr.div = div
cs.text = narr


#print(dumps(cs.as_json(),indent=3))

Resource Type,Supported Profiles,Supported Searches,Supported Includes,Supported Operations
AllergyIntolerance,US Core AllergyIntolerance Profile,"patient, patient+clinical-status",,
CarePlan,US Core CarePlan Profile,", patient+category, patient+category+status, patient+category+status+date, patient+category+date",,
CareTeam,US Core CareTeam Profile,", patient+status",,
Condition,US Core Condition Profile,"patient, patient+onset-date, patient+category, patient+code, patient+clinical-status",,
Device,US Core Device Profile,"patient, patient+type",,
DiagnosticReport,"US Core DiagnosticReport Profile for Report and Note exchange, US Core DiagnosticReport Profile for Laboratory Results Reporting",", patient+code+date, patient+code, patient+status, patient+category, patient+category+date, patient+category, patient+category+date",,
DocumentReference,US Core DocumentReference Profile,"_id, patient, patient+status, patient+category, patient+category+date, patient+type+period, patient+type",,$docref
Encounter,US Core Encounter Profile,"_id, identifier, patient, patient+status, class+patient, patient+type, date+patient",,
Goal,US Core Goal Profile,"patient, patient+target-date, patient+lifecycle-status",,
Immunization,US Core Immunization Profile,"patient, patient+date, patient+status",,


### validate again

In [154]:
print('...validating')
r = validate(cs)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))


...validating


### Write to folder

In [155]:
# save to file
print('...........saving to file............')
#save in ig_source folder
path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs.id.lower()}.json'
path.write_text(dumps(cs.as_json(), indent=4))

...........saving to file............


258643